<a href="https://colab.research.google.com/github/AwuahDarko/plant-pathology-CNN/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, LSTM, Bidirectional, Dropout, Flatten, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
import datetime
from datetime import datetime as dt, timedelta
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import confusion_matrix
import itertools
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import RobustScaler
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Added because of this error
# Error #15: Initializing libiomp5.dylib, but found libomp.dylib already initialized.
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ['CUDA_VISIBLE_DEVICES'] = 'True'
# =============================================

# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_booking = pd.read_csv('/content/drive/MyDrive/cab_booking.csv',low_memory=False)

In [4]:
df_request = pd.read_csv('/content/drive/MyDrive/cab_request_now.csv',low_memory=False)

In [5]:
df_booking.head()

,iCabBookingId,vBookingNo,iCompanyId,iUserId,iDriverId,iOrganizationId,vSourceLatitude,vSourceLongitude,vDestLatitude,vDestLongitude,vDistance,vDuration,dBooking_date,vSourceAddresss,tDestAddress,dAddedDate,iTripId,eStatus,ePayType,iVehicleTypeId,iRentalPackageId,fPickUpPrice,fNightPrice,eCancelBy,iCancelByUserId,dCancelDate,iCancelReasonId,vFailReason,vCancelReason,eType,iPackageTypeId,vReceiverName,vReceiverMobile,tPickUpIns,tDeliveryIns,tPackageDetails,vCouponCode,iUserPetId,eMessageSend,eMessageAdminOne,...,vDiscount,fMinFareDiff,fWalletDebit,fTax1,fTax2,fTax1Percentage,fTax2Percentage,fOutStandingAmount,iAdminId,iHotelBookingId,vRiderRoomNubmer,eBookingFrom,eWalletDebitAllow,ePaymentBy,vProfileEmail,iUserProfileId,iTripReasonId,vReasonTitle,eTripReason,fAirportPickupSurge,fAirportDropoffSurge,eBookForSomeOneElse,vBookSomeOneName,vBookSomeOneNumber,ePayWallet,eServiceLocation,tVehicleTypeData,tVehicleTypeFareData,tTotalDuration,tTotalDistance,tEstimatedCharge,tUserWalletBalance,vWorkLocation,vWorkLocationLatitude,vWorkLocationLongitude,eSelectWorkLocation,tReceiverPhoneCode,tPickupPhoneCode,tPickupPhoneNumber,wSenderContactedBy
0,1,84361311,1,13,4,0,5.534415,-0.425274,5.637614,-0.126606,39,61,3/7/2020 22:39,"Kasoa, Ghana","Spintex Road, Accra, Ghana",3/7/2020 17:08,4,Completed,Cash,3,0,1,1.0,NaN,0,0000-00-00 00:00:00,0,NaN,NaN,Deliver,2,Kwesi Emma,205839721,I'm wearing a white top,deliver it to the store,food,NaN,0,No,No,...,NaN,0,0,0,0,0,0,0,0,0,NaN,Admin,Yes,Passenger,NaN,0,0,NaN,No,0,0,No,NaN,NaN,No,Passanger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dynamic,NaN,NaN,NaN,None
1,2,74918494,1,13,61,0,5.648731,-0.226836,5.602612,-0.079195,28,47,3/16/2020 17:15,"Pillar 2 Assemblies of God Church, Unnamed Roa...","Accra - Tema Beach Road, Tema, Ghana",3/15/2020 11:43,0,Assign,Cash,3,0,1,1.0,NaN,0,0000-00-00 00:00:00,0,NaN,NaN,Deliver,10,Kwesi Emma,241989930,I'm wearing a white top,deliver it to the store,clothe,NaN,0,Yes,Yes,...,NaN,0,0,0,0,0,0,0,0,0,NaN,Admin,Yes,Passenger,NaN,0,0,NaN,No,0,0,No,NaN,NaN,No,Passanger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dynamic,NaN,NaN,NaN,None
2,3,95635906,1,46,66,0,5.550347,-0.206521,5.677242,-0.023127,29,42,3/18/2020 20:30,"Central Lorry station, Accra, Ghana","Tema General Hospital, Tema, Ghana",3/18/2020 13:48,10,Completed,Cash,3,0,1,1.0,NaN,0,0000-00-00 00:00:00,0,NaN,NaN,Deliver,10,Ewura Esi,240949836,call eric 0278065388 to pick up item,call receiver when at destination,polo shirts 5,NaN,0,No,Yes,...,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,No,Passenger,NaN,0,0,NaN,No,0,0,No,NaN,NaN,No,Passanger,NaN,NaN,42.47,29.39,NaN,NaN,NaN,NaN,NaN,Dynamic,NaN,NaN,NaN,None
3,4,20878750,1,119,66,0,5.571529,-0.146461,5.635828,-0.161359,12,20,3/19/2020 22:30,"Ghana International Trade Fair Centre, La Rd, ...","East Legon, Accra, Ghana",3/19/2020 16:29,14,Completed,Cash,3,0,1,1.0,NaN,0,0000-00-00 00:00:00,0,NaN,NaN,Deliver,11,East legon,244038590,Call her when you get there,call her for instructions,other,NaN,0,No,Yes,...,NaN,0,0,0,0,0,0,0,0,0,NaN,Admin,Yes,Passenger,NaN,0,0,NaN,No,0,0,No,NaN,NaN,No,Passanger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dynamic,NaN,NaN,NaN,None
4,5,16055725,1,119,66,0,5.571529,-0.146461,5.570663,-0.284979,20,39,3/19/2020 23:30,"Ghana International Trade Fair Centre, La Rd, ...","Mallam Junction Station, Gbawe Road, Mallam, G...",3/19/2020 16:44,0,Assign,Cash,3,0,1,1.0,NaN,0,0000-00-00 00:00:00,0,NaN,NaN,Deliver,11,Augesta,548278376,call the number when you get there,deliver to her house,others,NaN,0,Yes,Yes,...,NaN,0,0,0,0,0,0,0,0,0,NaN,Admin,Yes,Passenger,NaN,0,0,NaN,No,0,0,No,NaN,NaN,No,Passanger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dynamic,NaN,NaN,NaN,None


In [6]:
booking_columns_to_drop = [
    "iDriverId",
    "iCompanyId",
    "iOrganizationId",
    "iTripId",
    "iRentalPackageId",
    "iVehicleTypeId",
    "iCancelByUserId",
    "iPackageTypeId",
    "vReceiverMobile",
#     "tPickUpIns",
#     "tDeliveryIns",
#     "tPackageDetails",
    "vCouponCode",
    "iUserPetId",
    "eMessageSend",
    "eMessageAdminOne",
    "eMessageAdminTwo",
    "iCronStage",
    "iUserAddressId",
    "tUserComment",
    "iTempTripDeliveryLocationId",
    "iAirportLocationId",
    "fTax1",
    "fTax2",
    "fTax1Percentage",
    "fTax2Percentage",
    "iAdminId",
    "iHotelBookingId",
    "vRiderRoomNubmer",
    "vProfileEmail",
    "iUserProfileId",
    "iTripReasonId",
    "vReasonTitle",
    "eTripReason",
    "vWorkLocation",
    "vWorkLocationLatitude",
    "vWorkLocationLongitude",
    "eSelectWorkLocation",
    "tReceiverPhoneCode",
    "tPickupPhoneCode",
    "tPickupPhoneNumber",
    "iUserId",
    "eHandiCapAccessibility",
    "eHandiCapAccessibility",
    "eCommisionDeductEnable",
    "vBookSomeOneName",
    "vBookSomeOneNumber",
    "iCabBookingId",
    "vSourceAddresss",
    "tDestAddress",
    "vSourceLatitude",
    "vSourceLongitude",
    "vDestLatitude",
    "vSourceLongitude",
    "iCancelReasonId",
    "fDriverDiscount",
    "dCancelDate",
    "eCancelBy",
    "vDistance",
    "vDuration",
    "vDestLongitude",
    "ePayType",
    "ePayType",
    "fPickUpPrice",
    "fNightPrice",
    "vFailReason",
    "vCancelReason",
#     "vReceiverName",
    "eAutoAssign",
    "iQty",
    "fVisitFee",
    "fMaterialFee",
    "fMiscFee",
    "vRideCountry",
    "fTollPrice",
    "vTollPriceCurrencyCode",
    "eTollSkipped",
    "eFemaleDriverRequest",
    "vTimeZone",
    "fWalletMinBalance",
    "fWalletBalance",
    "ePaymentByReceiver",
    "fTripGenerateFare",
    "eCancelBySystem",
    "eFlatTrip",
    "fFlatTripPrice",
    "eAirportLocation",
    "iFare",
    "iBaseFare",
    "fPricePerMin",
    "fPricePerKM",
    "fCommision",
    "fSurgePriceDiff",
    "fDiscount",
    "vDiscount",
    "fMinFareDiff",
    "fOutStandingAmount",
    "eBookingFrom",
    "eWalletDebitAllow",
    "ePaymentBy",
    "fAirportPickupSurge",
    "fAirportDropoffSurge",
    "eBookForSomeOneElse",
    "ePayWallet",
    "eServiceLocation",
    "tVehicleTypeData",
    "tVehicleTypeFareData",
    "tTotalDuration",
    "tTotalDistance",
    "tEstimatedCharge",
    "tUserWalletBalance",
    "eAssigned",
    "fWalletDebit",
    "vBookingNo",
    "eStatus",
    "dBooking_date",
    "wSenderContactedBy",
   "dAddedDate",
    "eType"
]

In [7]:
request_columns_to_drop = [
    "iCabRequestId",
    "fAirportPickupSurge",
    "fAirportDropoffSurge",
    "iCabBookingId",
    "iUserId",
    "iDriverId",
    "iOrganizationId",
    "iHotelId",
    "tMsgCode",
    "iTripId",
    "vSourceLatitude",
    "vSourceLongitude",
    "tSourceAddress",
    "vDestLatitude",
    "vDestLongitude",
    "tDestAddress",
    "ePayType",
    "iVehicleTypeId",
    "iRentalPackageId",
    "fPickUpPrice",
    "fNightPrice",
    "iPackageTypeId",
#     "vReceiverName",
    "vReceiverMobile",
#     "tPickUpIns",
#     "tDeliveryIns",
#     "tPackageDetails",
    "vCouponCode",
    "iQty",
    "vRideCountry",
    "fTollPrice",
    "vTollPriceCurrencyCode",
    "eTollSkipped",
    "eFemaleDriverRequest",
    "eHandiCapAccessibility",
    "vTimeZone",
    "eFromCronJob",
    "iUserAddressId",
    "tUserComment",
    "iTempTripDeliveryLocationId",
    "ePaymentByReceiver",
    "fDuration",
    "fDistance",
    "fTripGenerateFare",
    "eFlatTrip",
    "fFlatTripPrice",
    "iAirportLocationId",
    "eAirportLocation",
    "iFare",
    "iBaseFare",
    "fPricePerMin",
    "fPricePerKM",
    "fCommision",
    "fSurgePriceDiff",
    "fDiscount",
    "vDiscount",
    "fMinFareDiff",
    "fWalletDebit",
    "fTax1",
    "fTax2",
    "fTax1Percentage",
    "fTax2Percentage",
    "fOutStandingAmount",
    "eBookingFrom",
    "iHotelBookingId",
    "eWalletDebitAllow",
    "eSystem",
    "ePaymentBy",
    "vProfileEmail",
    "iUserProfileId",
    "iTripReasonId",
    "vReasonTitle",
    "eTripReason",
    "ePoolRide",
    "iPersonSize",
    "eBookForSomeOneElse",
    "vBookSomeOneName",
    "vBookSomeOneNumber",
    "ePayWallet",
    "eServiceLocation",
    "tVehicleTypeData",
    "tVehicleTypeFareData",
    "tTotalDuration",
    "tTotalDistance",
    "tEstimatedCharge",
    "tUserWalletBalance",
    "tEditHistory",
    "tReceiverPhoneCode",
    "tPickupPhoneCode",
    "tPickupPhoneNumber",
    "eStatus",
    "dAddedDate",
    "eType"
]

In [8]:
df_booking.drop(booking_columns_to_drop,  axis=1, inplace=True)

In [9]:
df_booking.head()

,vReceiverName,tPickUpIns,tDeliveryIns,tPackageDetails
0,Kwesi Emma,I'm wearing a white top,deliver it to the store,food
1,Kwesi Emma,I'm wearing a white top,deliver it to the store,clothe
2,Ewura Esi,call eric 0278065388 to pick up item,call receiver when at destination,polo shirts 5
3,East legon,Call her when you get there,call her for instructions,other
4,Augesta,call the number when you get there,deliver to her house,others


In [ ]:
df_booking.tail()

,vReceiverName,tPickUpIns,tDeliveryIns,tPackageDetails
9998,La Tua Villas,pick up from Obojo warehouse storey building t...,near SDA Church Adenta,5 boxes of sanitizer
9999,jtuty,urgent,urgent,hair
10000,na,please pickup as soon as possible,please deliver on time it urgent,mobile phone
10001,Mavis,Abokobi,To Mavis,Rice milk mackereal T roll
10002,Mary,Please pick up early,Its a short program so delivery should be made...,Gift


In [11]:
df_request.drop(request_columns_to_drop,  axis=1, inplace=True)

In [12]:
df_request.head()

,vReceiverName,tPickUpIns,tDeliveryIns,tPackageDetails
0,tedt,uccug,cuvuvuv,ycyvubub8bib8
1,tedt,uccug,cuvuvuv,ycyvubub8bib8
2,Kofi,pick up,drop,jollof
3,Nana kay,yh,drop,phone
4,eunice,pick up from patience,deliver only to Eunice,delicate


In [ ]:
df_request.tail()

,vReceiverName,tPickUpIns,tDeliveryIns,tPackageDetails
22213,Rhida,0272980544,cable,cable
22214,Stephanie,lakeside,pokuasi,clothes
22215,Stephanie,lakeside,pokuasi,clothes
22216,Addo Benjamin,SpeedSmoke Services 0203668614 Call him,Ablekuma Fan Milk,Medicine
22217,Addo Benjamin,SpeedSmoke Services 0203668614 Call him,Ablekuma Fan Milk,Medicine


In [13]:
df = df_request.append(df_booking)

In [ ]:
df.head(10)

,vReceiverName,tPickUpIns,tDeliveryIns,tPackageDetails
0,tedt,uccug,cuvuvuv,ycyvubub8bib8
1,tedt,uccug,cuvuvuv,ycyvubub8bib8
2,Kofi,pick up,drop,jollof
3,Nana kay,yh,drop,phone
4,eunice,pick up from patience,deliver only to Eunice,delicate
5,Abel,call,drop,docs
6,Abel,call,deop,docs
7,Emma,call,drop,cloth
8,Abel,at the office,drop,phone
9,Joseph Abizi,Take money,Take money,y


In [14]:
df = df.dropna()

In [15]:
df.isnull().any()

vReceiverName      False
tPickUpIns         False
tDeliveryIns       False
tPackageDetails    False
dtype: bool

In [ ]:
df.head()

,vReceiverName,tPickUpIns,tDeliveryIns,tPackageDetails
0,tedt,uccug,cuvuvuv,ycyvubub8bib8
1,tedt,uccug,cuvuvuv,ycyvubub8bib8
2,Kofi,pick up,drop,jollof
3,Nana kay,yh,drop,phone
4,eunice,pick up from patience,deliver only to Eunice,delicate


In [16]:
tokenizer = Tokenizer()
corpus = []

In [17]:
# corpus.append(x for x in list(df['tPickUpIns']))
for x in list(df['tPickUpIns']):
    corpus.append(x)

In [18]:
# corpus.append(x for x in list(df['tDeliveryIns']))
for x in list(df['tDeliveryIns']):
    corpus.append(x)

In [ ]:
len(corpus)

58934

In [19]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 

In [ ]:
total_words

10146

In [20]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [21]:
max_sequence_len = max([len(x) for x in input_sequences])

In [22]:
max_sequence_len

57

In [23]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
input_sequences

array([[  0,   0,   0, ...,   0,   4,   5],
       [  0,   0,   0, ...,   0,   4,   5],
       [  0,   0,   0, ...,   4,   5,  13],
       ...,
       [  0,   0,   0, ...,  12,  57,  20],
       [  0,   0,   0, ...,  57,  20, 241],
       [  0,   0,   0, ...,  20, 241, 399]], dtype=int32)

In [24]:
xs = input_sequences[:,:-1]

In [25]:
labels = input_sequences[:,-1]

In [26]:
ys = to_categorical(labels, num_classes=total_words)

In [27]:
model = tf.keras.Sequential([
    Embedding(total_words, 30, input_length=max_sequence_len-1),
    Bidirectional(LSTM(150, return_sequences=True )),
    LSTM(units=30 ),
    Dense(units=320, activation='relu'), 
    Dense(total_words, activation='softmax'),
])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 56, 30)            304380    
_________________________________________________________________
bidirectional (Bidirectional (None, 56, 300)           217200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                39720     
_________________________________________________________________
dense (Dense)                (None, 320)               9920      
_________________________________________________________________
dense_1 (Dense)              (None, 10146)             3256866   
Total params: 3,828,086
Trainable params: 3,828,086
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy']) 

In [33]:
history = model.fit(xs, ys, epochs=60, batch_size=2500) # 30  

Epoch 1/10
60/60 [==============================] - 454s 8s/step - loss: 1.2758 - accuracy: 0.7011
Epoch 2/10
60/60 [==============================] - 450s 8s/step - loss: 1.2662 - accuracy: 0.7022
Epoch 3/10
60/60 [==============================] - 453s 8s/step - loss: 1.2583 - accuracy: 0.7048
Epoch 4/10
60/60 [==============================] - 478s 8s/step - loss: 1.2485 - accuracy: 0.7060
Epoch 5/10
60/60 [==============================] - 478s 8s/step - loss: 1.2398 - accuracy: 0.7081
Epoch 6/10
60/60 [==============================] - 474s 8s/step - loss: 1.2382 - accuracy: 0.7079
Epoch 7/10
60/60 [==============================] - 471s 8s/step - loss: 1.2327 - accuracy: 0.7084
Epoch 8/10
60/60 [==============================] - 481s 8s/step - loss: 1.2398 - accuracy: 0.7075
Epoch 9/10
60/60 [==============================] - 480s 8s/step - loss: 1.2314 - accuracy: 0.7090
Epoch 10/10
60/60 [==============================] - 445s 7s/step - loss: 1.2135 - accuracy: 0.7132


In [1]:
# checks first to see if  file exists already
# if not, the model is saved to disk
import os.path 

if os.path.isfile('/content/drive/MyDrive/textgen_model2.h5') is False:
    model.save('/content/drive/MyDrive/textgen_model2.h5')

NameError: ignored

In [ ]:
seed_text = "it still inside"
next_words = 1

In [ ]:
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ''
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += ' ' + output_word

print(seed_text)

In [ ]:
all_words_as_one_string = ""
all_words_as_one_string += ' '.join(df.tDeliveryIns.to_numpy().tolist())
all_words_as_one_string += ' '.join(df.tPickUpIns.to_numpy().tolist())

In [ ]:
text_cloud = WordCloud(stopwords=STOPWORDS, background_color="white").generate(all_words_as_one_string)

In [ ]:
def show_word_cloud(cloud):
    plt.figure(figsize=(16, 10))
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()